## Time Series Analysis and Forecasting for Bitcoin (BTC) using ARIMA
### Executive Summary

This project employs time series analysis and forecasting techniques, specifically leveraging the ARIMA model, to gain insights into the historical trends and make predictions for Bitcoin (BTC) prices. The analysis covers data resampling, stationarity checks, Box-Cox transformations, differencing, SARIMA model selection, and forecasting.

#### Key Steps and Findings:

1. **Data Exploration and Resampling:**
   - Bitcoin price data is loaded and resampled at daily, monthly, quarterly, and annual frequencies, offering a comprehensive view of the price trends.


2. **Seasonal-Trend Decomposition using LOESS (STL):**
   - STL decomposition reveals the underlying components of the time series, including trend, seasonal, and residual aspects.


3. **Stationarity Analysis:**
   - Dickey–Fuller tests are applied before and after Box-Cox transformation, ensuring stationarity for more reliable modeling.


4. **Differencing for Stationarity:**
   - Seasonal and regular differencing are employed to achieve stationarity, allowing for more accurate modeling.


5. **SARIMA Model Selection:**
   - Grid search is conducted to identify the optimal parameters for the SARIMA model, using the AIC as a criterion for selection.


6. **Model Diagnostic and Residual Analysis:**
   - Residual analysis confirms the model's adequacy, and the Dickey–Fuller test validates the stationarity of the residuals.


7. **Forecasting:**
   - The SARIMA model is utilized to forecast Bitcoin prices into the future, providing insights into potential trends.


8. **Inverse Transformation and Final Forecast Visualization:**
   - Forecasts are transformed back to the original data scale using the inverse Box-Cox transformation, and the results are visualized alongside historical data.


#### Conclusion:

The ARIMA model has been successfully applied to capture patterns and dynamics within the Bitcoin price time series. The forecasting results provide valuable information for decision-making and risk management in the cryptocurrency market. 

## Data Loading and Initial Exploration

The historical data of Bitcoin (BTC) is downloaded from Yahoo Finance https://finance.yahoo.com/quote/BTC-USD/history?p=BTC-USD , This csv contains data from 2014-09-17 to 2023-11-17.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
import statsmodels.api as sm
import warnings
from itertools import product
from datetime import datetime
warnings.filterwarnings('ignore')
plt.style.use('seaborn-poster')


#Loading the BTC CSV data
btc_data = pd.read_csv('/kaggle/input/btc-usd-2014-2023/BTC-USD.csv')
btc_data

### Time Series Preprocessing: Resampling BTC Data

In [ ]:
#Ensuring the 'Date' column to datetime type for proper time series handling
btc_data['Date'] = pd.to_datetime(btc_data['Date'])

#Setting the 'Date' column as the index
btc_data.set_index('Date', inplace=True)

#Resample to daily, monthly, quarterly and annual frequency
btc_data_daily = btc_data.resample('D').mean()
btc_data_monthly = btc_data.resample('M').mean()
btc_data_quarterly = btc_data.resample('Q').mean()
btc_data_annual = btc_data.resample('Y').mean()

In [ ]:
#Plotting for each frequency
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))

btc_data_daily['Close'].plot(ax=axes[0, 0], title='Daily Resampling', legend=True)
btc_data_monthly['Close'].plot(ax=axes[0, 1], title='Monthly Resampling', legend=True)
btc_data_quarterly['Close'].plot(ax=axes[1, 0], title='Quarterly Resampling', legend=True)
btc_data_annual['Close'].plot(ax=axes[1, 1], title='Annual Resampling', legend=True)

plt.tight_layout()
plt.show()

### STL-decomposition and  Stationarity check of the series

In [ ]:
#STL decomposition
plt.figure(figsize=[15, 7])
result = sm.tsa.seasonal_decompose(btc_data_monthly['Close'], period=12)
result.plot()

plt.show()

#Dickey-Fuller test results
adf_result = sm.tsa.adfuller(btc_data_monthly['Close'])
print("Dickey–Fuller test before transformation:")
print("ADF Statistic:", adf_result[0])
print("p-value:", adf_result[1])
print("Critical Values:", adf_result[4])

#Checking stationarity
if adf_result[1] <= 0.05:
    print("The time series is likely stationary (reject the null hypothesis)")
else:
    print("The time series is likely non-stationary (fail to reject the null hypothesis)")

### Box-Cox Transformations

In [ ]:
#Box-Cox transformation
btc_data_monthly['Close_box'], lmbda = stats.boxcox(btc_data_monthly['Close'])

#Checking stationarity
result_after = sm.tsa.adfuller(btc_data_monthly['Close_box'])

print("\nDickey–Fuller test after transformation:")
print("ADF Statistic:", result_after[0])
print("p-value:", result_after[1])
print("Critical Values:", result_after[4])
if result_after[1] <= 0.05:
    print("The transformed time series is likely stationary (reject the null hypothesis)")
else:
    print("The transformed time series is likely non-stationary (fail to reject the null hypothesis)")

### Seasonal Differentiation

In [ ]:
#Seasonal differencing
btc_data_monthly['prices_box_diff'] = btc_data_monthly['Close_box'] - btc_data_monthly['Close_box'].shift(12)

#Droping NaN values resulting from the differencing
btc_data_monthly.dropna(inplace=True)

#Checking Stationarity
adf_result_diff = sm.tsa.adfuller(btc_data_monthly['prices_box_diff'])

print("Dickey–Fuller test after seasonal differencing:")
print("ADF Statistic:", adf_result_diff[0])
print("p-value:", adf_result_diff[1])
print("Critical Values:", adf_result_diff[4])
if adf_result_diff[1] <= 0.05:
    print("The differenced time series is likely stationary (reject the null hypothesis)")
else:
    print("The differenced time series is likely non-stationary (fail to reject the null hypothesis)")

### Regular Differentiation

In [ ]:
#Regular differencing
btc_data_monthly['prices_box_diff2'] = btc_data_monthly['prices_box_diff'] - btc_data_monthly['prices_box_diff'].shift(1)

#Droping NaN values resulting from the differencing
btc_data_monthly.dropna(inplace=True)

#Plotting STL decomposition
plt.figure(figsize=(15, 7))
sm.tsa.seasonal_decompose(btc_data_monthly['prices_box_diff2']).plot()

plt.show()

#Checking Stationarity
adf_result_diff2 = sm.tsa.adfuller(btc_data_monthly['prices_box_diff2'])

print("Dickey–Fuller test after regular differencing:")
print("ADF Statistic:", adf_result_diff2[0])
print("p-value:", adf_result_diff2[1])
print("Critical Values:", adf_result_diff2[4])
if adf_result_diff2[1] <= 0.05:
    print("The doubly differenced time series is likely stationary (reject the null hypothesis)")
else:
    print("The doubly differenced time series is likely non-stationary (fail to reject the null hypothesis)")

### Model Selection

In [ ]:
#Autocorrelation and Partial Autocorrelation Plots
plt.figure(figsize=(15, 7))
ax1 = plt.subplot(211)
sm.graphics.tsa.plot_acf(btc_data_monthly['prices_box_diff2'][13:].values.squeeze(), lags=48, ax=ax1)
ax2 = plt.subplot(212)
sm.graphics.tsa.plot_pacf(btc_data_monthly['prices_box_diff2'][13:].values.squeeze(), ax=ax2)  # Using the default number of lags
plt.tight_layout()
plt.show()

In [ ]:
#Initial approximation of parameters
Qs = range(0, 2)
qs = range(0, 3)
Ps = range(0, 3)
ps = range(0, 3)
D = 1
d = 1
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
print(f"Total parameter combinations: {len(parameters_list)}")

#Model Selection
results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')

for i, param in enumerate(parameters_list, start=1):
    print(f"Fitting model {i}/{len(parameters_list)} with parameters: {param}")
    try:
        model = sm.tsa.statespace.SARIMAX(
            btc_data_monthly['Close_box'],
            order=(param[0], d, param[1]),
            seasonal_order=(param[2], D, param[3], 12)
        ).fit(disp=-1)
    except Exception as e:
        print(f"Error fitting model with parameters {param}: {e}")
        continue

    aic = model.aic
    results.append([param, aic])

    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param

print(f"\nBest model AIC: {best_aic}")
print(f"Best model parameters: {best_param}")

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by = 'aic', ascending=True).head())
print(best_model.summary())

### Analysis of Residues

In [ ]:
#STL-decomposition and Residual Analysis
plt.figure(figsize=(15, 7))

#Plotting Residuals
plt.subplot(2, 1, 1)
best_model.resid[13:].plot()
plt.ylabel('Residuals')

#Plotting ACF of Residuals
ax = plt.subplot(2, 1, 2)
sm.graphics.tsa.plot_acf(best_model.resid[13:].values.squeeze(), lags=48, ax=ax)
plt.tight_layout()
plt.show()

# Dickey–Fuller test for stationarity
print("Dickey–Fuller test: p=%f" % sm.tsa.stattools.adfuller(best_model.resid[13:])[1])

## Prediction
We will predict bitcoin prices by months.

In [ ]:
#Number of periods to forecast into the future
forecast_periods = 12

#Forecast
forecast = best_model.get_forecast(steps=forecast_periods)

#Extractting forecasted mean and confidence intervals
forecast_mean = forecast.predicted_mean
forecast_conf_int = forecast.conf_int()

#Assuming your data has a datetime index
forecast_index = pd.date_range(start=btc_data_monthly.index[-1] + pd.DateOffset(months=1), periods=forecast_periods, freq='M')

#Plotting the observed values, the fitted values, and the forecast
plt.figure(figsize=(15, 7))
plt.plot(btc_data_monthly['Close_box'], label='Observed')
plt.plot(best_model.fittedvalues, label='Fitted', color='orange')
plt.plot(forecast_index, forecast_mean, label='Forecast', color='green')
plt.fill_between(forecast_index, forecast_conf_int.iloc[:, 0], forecast_conf_int.iloc[:, 1], color='green', alpha=0.3)

plt.title('ARIMA Model Forecast')
plt.xlabel('Date')
plt.ylabel('Box-Cox Transformed Weighted Price')
plt.legend()
plt.show()

In [ ]:
#Inverse Box-Cox Transformation Function
def invboxcox(y,lmbda):
    if lmbda == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(lmbda*y+1)/lmbda))

#Creating a new column with the original Close values (inverse Box-Cox if applied)
btc_data_monthly['Original_Close'] = invboxcox(btc_data_monthly['Close_box'], lmbda)

#Number of periods to forecast into the future
forecast_periods = 12

#Forecast
forecast = best_model.get_forecast(steps=forecast_periods)

#Extractting forecasted mean and confidence intervals
forecast_mean = forecast.predicted_mean
forecast_conf_int = forecast.conf_int()

#Datetime index
forecast_index = pd.date_range(start=btc_data_monthly.index[-1] + pd.DateOffset(months=1), periods=forecast_periods, freq='M')

#Inverse transforming the forecasted values
forecasted_usd = invboxcox(forecast_mean, lmbda)

#Plotting USD values against years for both actual and predicted
plt.figure(figsize=(15, 7))
plt.plot(btc_data_monthly.index, btc_data_monthly['Original_Close'], label='Actual USD Values', color='blue')
plt.plot(forecast_index, forecasted_usd, label='Predicted USD Values', color='red')
plt.fill_between(forecast_index, invboxcox(forecast_conf_int.iloc[:, 0], lmbda), invboxcox(forecast_conf_int.iloc[:, 1], lmbda), color='red', alpha=0.3)

plt.title('Bitcoin USD Values Over Time - Actual vs Predicted')
plt.xlabel('Years')
plt.ylabel('USD Values')
plt.legend()
plt.show()

In [ ]:
#Creating a DataFrame to store the predicted values and corresponding dates
predicted_df = pd.DataFrame({'Predicted_USD': forecasted_usd})
print(predicted_df)